In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service

In [11]:
import requests
from bs4 import BeautifulSoup
import json
import sys

In [12]:
import pandas as pd

In [13]:
def myHTMLParser(s):
    i = s.find('href=\"')
    newString = s[i+6:]
    i2 = newString.find('\"')
    return newString[:i2]

In [14]:
# driver.page_source

In [35]:
def scrapeLocation(url):
    SER = Service('/Users/jimmydelano/Downloads/chromedriver')
    OP = webdriver.ChromeOptions()
    # url = 'https://www.lennar.com/new-homes/alabama/coastal-alabama'
    driver = webdriver.Chrome(service=SER, options=OP)
    driver.get(url)
    driver.maximize_window()
    time.sleep(10)


    try:
        driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div/button").click()
    except:
        print("No cookies window. Continue...")

    time.sleep(5)

    ## Click the load more homes buttons as many times as necessary...
    more_homes = True
    while more_homes:
        try:
            load_more = driver.find_element(By.XPATH, '//button[normalize-space()="Load more homes"]')
            driver.execute_script("arguments[0].click();", load_more)
            time.sleep(5)
        except:
            print("No more homes to load... Scraping text")
            more_homes = False

    ## scrape that data
    keep_scraping = True
    index = 1
    data = []
    while keep_scraping:

        try:
            result = driver.find_element(By.XPATH, 
                            f'//*[@id="main"]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[{index}]').text.splitlines()

            ## skip over the Ad
            if index == 2 and 'Year-End Sales Event' in result[0]:
                index += 1
                continue

            ## flex-cash thing -- just drop it
            if result[0] == 'Flex Cash':
                result = result[1:]

            ## reduced price
            if result[1][0] == '$' and result[2][0] == '$':
                result.pop(2)

            ## weird thing where sometimes a half bath is a separate column.
            ## grab that and add it to the total baths (divide by 2)
            if 'half ba' in result[4]:
                i = int(result[4][0]) / 2
                b = int(result[3][0])
                result[3] = str(i+b) + " ba"
                result.pop(4)


            assert len(result) == 7, "**** Some other data format issue ****"

            ## Rearranging columns for data
            if result[0] == "Future release":
                result.pop(1)
                result.insert(5, "N/A")

            ## get link
            html = driver.find_element(By.XPATH, 
                            f'//*[@id="main"]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[{index}]').get_attribute('innerHTML')
            url = 'https://www.lennar.com' + myHTMLParser(html)
            result.append(url)

            data.append(result)
            index += 1
        except:
            print("Done Scraping info")
            keep_scraping = False
    
    
    
    df = pd.DataFrame(data, columns = ["Availability", "Price", "Beds", "Baths", "Sqft", "Address", "Community", "URL"])
    df['Beds'] = df['Beds'].apply(lambda x : float(x.replace(" bd","")))
    df['Baths'] = df['Baths'].apply(lambda x : float(x.replace(" ba","")))
    df['Sqft'] = df['Sqft'].apply(lambda x : float(x.replace(" ft²", "").replace(",","")))
    df['Price'] = df.Price.replace('\D', '', regex=True).astype(int)
    df['Price'] = df.apply(lambda row: row['Price'] if row['Availability'] != 'Future release' 
                           else row['Price']*1000, axis=1)
    return df

In [36]:
df1 = scrapeLocation('https://www.lennar.com/new-homes/alabama/huntsville')

No more homes to load... Scraping text
Done Scraping info


ValueError: Columns must be same length as key

In [40]:
SER = Service('/Users/jimmydelano/Downloads/chromedriver')
OP = webdriver.ChromeOptions()
url = 'https://www.lennar.com/new-homes/alabama/coastal-alabama'
driver = webdriver.Chrome(service=SER, options=OP)
driver.get(url)
driver.maximize_window()
time.sleep(10)


try:
    driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div/button").click()
except:
    print("No cookies window. Continue...")

time.sleep(5)

## Click the load more homes buttons as many times as necessary...
more_homes = True
while more_homes:
    try:
        load_more = driver.find_element(By.XPATH, '//button[normalize-space()="Load more homes"]')
        driver.execute_script("arguments[0].click();", load_more)
        time.sleep(5)
    except:
        print("No more homes to load... Scraping text")
        more_homes = False

## scrape that data
keep_scraping = True
index = 1
data = []
while keep_scraping:

    try:
        result = driver.find_element(By.XPATH, 
                        f'//*[@id="main"]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[{index}]').text.splitlines()

        ## skip over the Ad
        if index == 2 and 'Year-End Sales Event' in result[0]:
            index += 1
            continue

        ## flex-cash thing -- just drop it
        if result[0] == 'Flex Cash':
            result = result[1:]

        ## reduced price
        if result[1][0] == '$' and result[2][0] == '$':
            result.pop(2)

        ## weird thing where sometimes a half bath is a separate column.
        ## grab that and add it to the total baths (divide by 2)
        if 'half ba' in result[4]:
            i = int(result[4][0]) / 2
            b = int(result[3][0])
            result[3] = str(i+b) + " ba"
            result.pop(4)


        assert len(result) == 7, "**** Some other data format issue ****"

        ## Rearranging columns for data
        if result[0] == "Future release":
            result.pop(1)
            result.insert(5, "N/A")

        ## get link
        html = driver.find_element(By.XPATH, 
                        f'//*[@id="main"]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[{index}]').get_attribute('innerHTML')
        url = 'https://www.lennar.com' + myHTMLParser(html)
        result.append(url)

        data.append(result)
        index += 1
    except:
        print("Done Scraping info")
        keep_scraping = False

No more homes to load... Scraping text
Done Scraping info


In [41]:
data

[]

In [42]:
df1

,Availability,Price,Beds,Baths,Sqft,Address,Community,URL
0,Move-in ready,341650,3.0,2.0,1846.0,"14961 Demaire Ln, Athens, AL",Residence 1846 in Browns Crossing West,https://www.lennar.com/new-homes/alabama/hunts...
1,Move-in ready,327900,3.0,2.0,1911.0,"14960 Demarie Ln, Athens, AL",Residence 1911 in Browns Crossing West,https://www.lennar.com/new-homes/alabama/hunts...
2,Move-in ready,409950,3.0,2.5,2369.0,"308 Hay Rake Dr, Madison, AL",Farmhouse Unit 6 in Clift Farm,https://www.lennar.com/new-homes/alabama/hunts...
3,Move-in ready,409950,3.0,2.5,2369.0,"306 Hay Rake Dr, Madison, AL",Farmhouse Unit 6 in Clift Farm,https://www.lennar.com/new-homes/alabama/hunts...
4,Move-in ready,318600,3.0,2.0,2033.0,"24212 Costello Dr, Athens, AL",Residence 2033 in Browns Crossing West,https://www.lennar.com/new-homes/alabama/hunts...
...,...,...,...,...,...,...,...,...
128,Future release,446000,4.0,3.0,3159.0,N/A,Residence 3159 in The Retreat,https://www.lennar.com/new-homes/alabama/hunts...
129,Future release,418000,4.0,3.0,2338.0,N/A,Residence 2338 in The Retreat,https://www.lennar.com/new-homes/alabama/hunts...
130,Future release,336000,3.0,2.0,1846.0,N/A,Residence 1846 in The Retreat,https://www.lennar.com/new-homes/alabama/hunts...
131,Future release,331000,3.0,2.0,1696.0,N/A,Residence 1696 in The Retreat,https://www.lennar.com/new-homes/alabama/hunts...


In [44]:
df1.to_csv('./huntsville-al.csv', index=False)

In [45]:
df2 = scrapeLocation("https://www.lennar.com/new-homes/alabama/coastal-alabama")

No more homes to load... Scraping text
Done Scraping info


In [46]:
df2

,Availability,Price,Beds,Baths,Sqft,Address,Community,URL
0,Move-in ready,431900,4.0,3.0,2828.0,"1739 Partridge ST, Foley, AL",Residence 2828 in Quail Landing,https://www.lennar.com/new-homes/alabama/coast...
1,Move-in ready,339900,3.0,2.5,2033.0,"1166 Pheasant Circle, Foley, AL",Residence 2033 in Quail Landing,https://www.lennar.com/new-homes/alabama/coast...
2,Under construction,366400,4.0,2.0,1815.0,"9815 Lakeview Dr, Foley, AL",Residence 1815 in Glenlakes,https://www.lennar.com/new-homes/alabama/coast...
3,Under construction,339900,3.0,2.5,2033.0,"1725 Grouse St, Foley, AL",Residence 2033 in Quail Landing,https://www.lennar.com/new-homes/alabama/coast...
4,Under construction,322900,4.0,2.0,1815.0,"1717 Grouse St, Foley, AL",Residence 1815 in Quail Landing,https://www.lennar.com/new-homes/alabama/coast...
5,Under construction,339900,3.0,2.5,2033.0,"1709 Grouse St, Foley, AL",Residence 2033 in Quail Landing,https://www.lennar.com/new-homes/alabama/coast...
6,Under construction,351900,4.0,3.0,2117.0,"1721 Grouse St, Foley, AL",Residence 2117 in Quail Landing,https://www.lennar.com/new-homes/alabama/coast...
7,Under construction,326400,4.0,2.0,1815.0,"1713 Grouse St, Foley, AL",Residence 1815 in Quail Landing,https://www.lennar.com/new-homes/alabama/coast...
8,Under construction,351400,4.0,3.0,2117.0,"1722 Grouse St, Foley, AL",Residence 2117 in Quail Landing,https://www.lennar.com/new-homes/alabama/coast...
9,Under construction,364900,3.0,2.0,1911.0,"9816 Lakeview Dr, Foley, AL",Residence 1911 in Glenlakes,https://www.lennar.com/new-homes/alabama/coast...


In [47]:
df2.to_csv('./coastal-al.csv', index=False)